In [1]:
import torch
import torchvision as tv
from PIL import Image
import numpy as np
import random
import tqdm
import torch.nn as nn
import torch.nn.functional as F

In [2]:
num_steps = 10000
batch_size = 128
device = 'cuda'

# Some usefull functions

In [3]:
def pimg(img):
  return Image.fromarray((img.detach().cpu().squeeze().numpy()*255).astype('uint8'))

In [4]:
class Calculator(nn.Module):
  def __init__(self):
    super(Calculator, self).__init__()
    self.encoder = nn.Sequential(
        nn.Linear(784*3, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256,20)
    )
    self.decoder = nn.Sequential(
        nn.Linear(10, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 784),
        nn.Sigmoid())
    
  def reparameterise(self, mu, logvar):
    if self.training:
        std = logvar.mul(0.5).exp_()
        eps = std.data.new(std.size()).normal_()
        return eps.mul(std).add_(mu)
    else:
        return mu

  def forward(self, x):
    batch_size, _ = x.shape
    mu_logvar = self.encoder(x).view(-1, 2, 10)
    mu = mu_logvar[:, 0, :]
    logvar = mu_logvar[:, 1, :]
    z = self.reparameterise(mu, logvar)
    return self.decoder(z).reshape([batch_size, 28, 28]), mu, logvar

# Creating the signs

In [5]:
minus = torch.zeros([1, 28, 28])
xcoord, ycoord = torch.meshgrid(torch.range(0, 27), torch.range(0, 27))
pidcs = (xcoord >= 12) & (xcoord < 16) 
minus[pidcs[None]] = 1
minus = minus[0]
pimg(minus)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  


In [6]:
plus = torch.zeros([1, 28, 28])
xcoord, ycoord = torch.meshgrid(torch.range(0, 27), torch.range(0, 27))
pidcs = (xcoord >= 6) & (xcoord < 22) & (ycoord >= 12) & (ycoord < 17)
plus[pidcs[None]] = 1
plus[pidcs.T[None]] = 1
plus = plus[0]
pimg(plus)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  


# Creating the dataset

In [ ]:
transormations = tv.transforms.Compose([tv.transforms.ToTensor()])
dataset = tv.datasets.MNIST(root='./data', download = True, transform = transormations)
y = [[d for d in dataset if d[1] == c] for c in range(10)]

In [8]:
pimg(dataset[1][0][0])

In [9]:
ds = [d for d in dataset]

# Training the model

In [10]:
netG = Calculator().to(device)
optimizerG = torch.optim.Adam(netG.parameters(), lr=3e-4)
mse_loss = nn.MSELoss()

In [11]:
for step in tqdm.trange(num_steps):
  batch = []
  labels = []
  optimizerG.zero_grad()
  segno=minus
  if step%2 ==0:
    segno = plus
  while len(batch) < batch_size:
    two1, two2 = np.random.choice(len(ds), size=2)
    first, second = ds[two1], ds[two2]
    tpt = torch.cat((first[0][0], segno, second[0][0]), 1).reshape(784*3).to(device)
    #tpt = first[0][0].reshape(784).to(device)
    l1 = first[1]
    l2 = second[1]
    label = l1 - l2
    if step%2 ==0:
      label = l1 + l2
      if label > 9:
        continue
    if label <0 :
        continue
    label_ds = y[label]
    label = random.choice(label_ds)[0][0].to(device)#first[0][0].to(device)
    batch.append(tpt)
    labels.append(label)
  batch = torch.stack(batch)
  labels = torch.stack(labels)
  output, mu, logvar = netG(batch)

  loss = mse_loss(labels, output)
  KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))
  #loss_total = loss + 2*KLD
  loss.backward()
  #loss_total.backward()
  optimizerG.step()
  if step % 999 == 0:
    print(loss, KLD)

  0%|          | 5/10000 [00:00<07:25, 22.43it/s]

tensor(0.2335, device='cuda:0', grad_fn=<MseLossBackward>) tensor(2.9932, device='cuda:0', grad_fn=<MulBackward0>)


 10%|█         | 1007/10000 [00:25<03:53, 38.54it/s]

tensor(0.0662, device='cuda:0', grad_fn=<MseLossBackward>) tensor(5300.3340, device='cuda:0', grad_fn=<MulBackward0>)


 20%|██        | 2003/10000 [00:50<03:23, 39.38it/s]

...

 90%|████████▉ | 8996/10000 [03:44<00:25, 40.03it/s]

tensor(0.0552, device='cuda:0', grad_fn=<MseLossBackward>) tensor(6068.4019, device='cuda:0', grad_fn=<MulBackward0>)


100%|█████████▉| 9997/10000 [04:09<00:00, 38.67it/s]

tensor(0.0551, device='cuda:0', grad_fn=<MseLossBackward>) tensor(6348.6758, device='cuda:0', grad_fn=<MulBackward0>)


100%|██████████| 10000/10000 [04:09<00:00, 40.13it/s]


# Test

In [12]:
test_dataset = tv.datasets.MNIST(root='./data', train=False, download=True,
                           transform=tv.transforms.Compose([
                               tv.transforms.ToTensor(),
                           ]))

In [53]:
test_ds = [d[0][0] for d in test_dataset]
test_two1, test_two2 = test_ds[100], test_ds[2]

In [55]:
test_tpt = torch.cat((test_two1, minus, test_two2), 1).to(device)
pimg(test_tpt)

In [59]:
test_tpt = test_tpt.unsqueeze(0).reshape([1, 784*3])
FINAL_OUTPUT = netG(test_tpt.to(device))[0]
pimg(FINAL_OUTPUT)